### An example implementation of spin vector component estimation from Statcast data. I'm referencing the following whitepaper by Dr. Alan nathan

#### https://baseball.physics.illinois.edu/HawkeyeAveSpinComponents.pdf

In [1]:
import sys
import numpy as np
import pandas as pd
import warnings
import plotly.graph_objects as go

sys.path.append('../src')
from fetch_savant import StatcastDataHandler

warnings.filterwarnings('ignore')


In [2]:

def calculate_air_density(altitude):

    sea_level_temp = 288.15
    sea_level_pressure = 101325
    lapse_rate = 0.0065
    gas_constant = 287.05

    temperature = sea_level_temp - lapse_rate * altitude

    pressure = sea_level_pressure * (1 - (lapse_rate * altitude / sea_level_temp)) ** (9.80665 / (gas_constant * lapse_rate))

    air_density = pressure / (gas_constant * temperature)

    return air_density

def estimate_k(altitude = None):

    sea_level_density=1.225
    k0 = 5.383E-03

    air_density = calculate_air_density(altitude)

    k = k0 * (sea_level_density / air_density)

    return k



In [3]:
def add_release_metrics(df,altitude_df):
    # Define the estimate_k() function
    z_constant = 32.174

    df = (
        df
        .merge(altitude_df,on = 'home_team',how = 'left')
        .assign(
            yR=lambda df: 60.5 - df['release_extension'],
            tR=lambda df: (-df['vy0'] - np.sqrt(df['vy0']**2 - 2 * df['ay'] * (50 - df['yR']))) / df['ay'],
            vxR=lambda df: df['vx0'] + df['ax'] * df['tR'],
            vyR=lambda df: df['vy0'] + df['ay'] * df['tR'],
            vzR=lambda df: df['vz0'] + df['az'] * df['tR'],
            tf=lambda df: (-df['vyR'] - np.sqrt(df['vyR']**2 - 2 * df['ay'] * (df['yR'] - 17/12))) / df['ay'],
            vxbar=lambda df: (2 * df['vxR'] + df['ax'] * df['tf']) / 2,
            vybar=lambda df: (2 * df['vyR'] + df['ay'] * df['tf']) / 2,
            vzbar=lambda df: (2 * df['vzR'] + df['az'] * df['tf']) / 2,
            vbar=lambda df: np.sqrt(df['vxbar']**2 + df['vybar']**2 + df['vzbar']**2),
            adrag=lambda df: -(df['ax'] * df['vxbar'] + df['ay'] * df['vybar'] + (df['az'] + z_constant) * df['vzbar']) / df['vbar'],
            amagx=lambda df: df['ax'] + df['adrag'] * df['vxbar'] / df['vbar'],
            amagy=lambda df: df['ay'] + df['adrag'] * df['vybar'] / df['vbar'],
            amagz=lambda df: df['az'] + df['adrag'] * df['vzbar'] / df['vbar'] + z_constant,
            amag=lambda df: np.sqrt(df['amagx']**2 + df['amagy']**2 + df['amagz']**2),
            K=lambda df: estimate_k(df['altitude']),
            Cl=lambda df: df['amag'] / (df['K'] * df['vbar']**2),
            s=lambda df: 0.166 * np.log(0.336 / (0.336 - df['Cl'])),
            spinT=lambda df: 78.92 * df['s'] * df['vbar'],
            spin_efficiency=lambda df: np.minimum(df['spinT'] / df['release_spin_rate'], 1)
        )
        .dropna(subset=['vx0','vy0','vz0','ax','ay','az','release_extension','release_spin_rate','spin_axis'])
        .reset_index(drop=True)
    )

    return(df)


In [4]:
def solve_spin_components_method_1(total_spin, phi, mean_cos_theta_s, vx, vy, vz):

    phi = np.radians((phi + 90) % 360)
    
    v = np.sqrt(vx**2 + vy**2 + vz**2)
    v_hat_x = vx / v
    v_hat_y = vy / v
    v_hat_z = vz / v
    
    A = v_hat_x * np.cos(phi) + v_hat_z * np.sin(phi)
    B = v_hat_y
    C = mean_cos_theta_s
    
    R = np.sqrt(A**2 + B**2)
    X = np.arctan2(B, A)
    
    # 3D theta
    Theta = np.arcsin(C / R) - X
    
    # Calculate spin components
    omega_x = total_spin * np.sin(Theta) * np.cos(phi)
    omega_y = total_spin * np.cos(Theta)
    omega_z = total_spin * np.sin(Theta) * np.sin(phi)
    
    return omega_x, omega_y, omega_z

In [5]:

def solve_spin_components_method_2(total_spin, phi, mean_cos_theta_s, vx, vy, vz):

    phi = np.radians(phi)
    
    # default is RHP. flips if LHP
    sign = 1 if mean_cos_theta_s < 0 else -1
    
    cos_Theta = sign * np.abs(1 - mean_cos_theta_s)
    
    # 3D theta
    Theta = np.arccos(cos_Theta)
    
    # spin components
    omega_x = total_spin * np.sin(Theta) * np.cos(phi)
    omega_y = total_spin * np.cos(Theta)
    omega_z = total_spin * np.sin(Theta) * np.sin(phi)
    
    return omega_x, omega_y, omega_z


In [6]:
def add_spin_components(df,method = 1):

    if method == 1:
        fun = solve_spin_components_method_1
    elif method == 2:
        fun = solve_spin_components_method_2
    else:
        raise ValueError('Method must be either 1 or 2')

    df['spin_x'], df['spin_y'], df['spin_z'] = zip(*df.apply(lambda row: fun(row['release_spin_rate'], row['spin_axis'], row['spin_efficiency'], row['vxR'], row['vyR'], row['vzR']), axis=1))


    return(df)

## Fetch Data
 
I'm importing locally but you should be able to use pybaseball scrapes directly

In [7]:

# load handler
handler = StatcastDataHandler()

#handler.update_local_sc(just_current=True)
# convenience function for fetching seasons, game types
season_data = handler.fetch_statcast(start_year=2024,game_types=['R'])


Fetching: [2024]


Grab Altitude data for estimating spin efficiency

In [8]:
csv_path = '../data/altitude_data.csv'
altitude_df = pd.read_csv(csv_path)

Calculate estimated spin efficiency, use 

In [9]:
season_data = add_release_metrics(season_data,altitude_df)


## Method 1

In [10]:
%%time
sc_m1 = add_spin_components(season_data,method=1)

CPU times: user 9.94 s, sys: 673 ms, total: 10.6 s
Wall time: 11.1 s


In [11]:
sc_m1.query('pitcher_name == "Gerrit Cole"').query('pitch_type == "FF"')[['pitcher_name','pitch_type','spin_x','spin_y','spin_z']].head(10)

,pitcher_name,pitch_type,spin_x,spin_y,spin_z
26329,Gerrit Cole,FF,1129.672454,-1339.639261,-1674.808287
26331,Gerrit Cole,FF,1052.408570,-1479.615268,-1560.259869
26333,Gerrit Cole,FF,732.628529,-1837.259305,-1268.949835
26334,Gerrit Cole,FF,957.603556,-1559.929854,-1593.719950
26336,Gerrit Cole,FF,1012.325066,-1464.891161,-1393.345918
26340,Gerrit Cole,FF,775.186870,-1613.151917,-1521.389894
26342,Gerrit Cole,FF,968.717785,-1723.964493,-1333.325646
26344,Gerrit Cole,FF,854.943943,-1350.116476,-1752.894850
26347,Gerrit Cole,FF,961.880742,-1411.517944,-1600.838384
26349,Gerrit Cole,FF,810.685621,-1502.553534,-1524.677902


# Method 2

In [12]:
%%time
sc_m2 = add_spin_components(season_data,method=2)

CPU times: user 8.07 s, sys: 475 ms, total: 8.55 s
Wall time: 8.69 s


In [13]:
sc_m2.query('pitcher_name == "Gerrit Cole"').query('pitch_type == "FF"')[['pitcher_name','pitch_type','spin_x','spin_y','spin_z']].head(10)

,pitcher_name,pitch_type,spin_x,spin_y,spin_z
26329,Gerrit Cole,FF,-1845.756660,-958.637925,-1244.978587
26331,Gerrit Cole,FF,-1882.179931,-759.530552,-1269.546394
26333,Gerrit Cole,FF,-2001.755947,-424.025357,-1155.714335
26334,Gerrit Cole,FF,-1989.233283,-710.389149,-1195.251942
26336,Gerrit Cole,FF,-1736.270644,-711.454445,-1261.474463
26340,Gerrit Cole,FF,-2002.269692,-684.027787,-1020.207365
26342,Gerrit Cole,FF,-1879.886887,-537.393811,-1365.817771
26344,Gerrit Cole,FF,-1969.014558,-909.444126,-960.352567
26347,Gerrit Cole,FF,-1878.587394,-822.842955,-1128.769185
26349,Gerrit Cole,FF,-1938.511593,-647.533777,-1030.724897


In [14]:
avg_dat = (
    sc_m2
    .groupby(['pitcher_name','pitcher','pitch_type'])
    .agg(
        spin_x = ('spin_x','mean'),
        spin_y = ('spin_y','mean'),
        spin_z = ('spin_z','mean'),
        vxR = ('vxR','mean'),
        vyR = ('vyR','mean'),
        vzR = ('vzR','mean')
    )
    .reset_index()
)

In [15]:
avg_dat.head()

,pitcher_name,pitcher,pitch_type,spin_x,spin_y,spin_z,vxR,vyR,vzR
0,A. J. Minter,621345,CH,-819.749258,-361.404125,1386.682533,-8.013780,-126.994041,-2.457543
1,A. J. Minter,621345,FC,-1239.442797,-2015.307452,-392.954844,-6.425901,-131.538719,-3.311777
2,A. J. Minter,621345,FF,-1898.801933,-876.517291,1163.777184,-6.993255,-138.623828,-5.079209
3,A. J. Puk,640462,CH,-830.527122,-248.587298,1743.375700,-11.642306,-131.712644,-3.565995
4,A. J. Puk,640462,FF,-1386.622989,-727.270787,1590.218243,-10.606709,-138.702478,-3.749648


In [16]:

pitcher_name = 'Gerrit Cole'
pitch_type = 'FF'

# select a pitcher, type
filtered_df = avg_dat.query('pitcher_name == @pitcher_name').query('pitch_type == @pitch_type')

spin_x = filtered_df['spin_x'].values[0]
spin_y = filtered_df['spin_y'].values[0]
spin_z = filtered_df['spin_z'].values[0]

vxR = filtered_df['vxR'].values[0]
vyR = filtered_df['vyR'].values[0]
vzR = filtered_df['vzR'].values[0]

spin_magnitude = np.sqrt(spin_x**2 + spin_y**2 + spin_z**2)

spin_x_normalized = spin_x / spin_magnitude
spin_y_normalized = spin_y / spin_magnitude
spin_z_normalized = spin_z / spin_magnitude

direction_magnitude = np.sqrt(vxR**2 + vyR**2 + vzR**2)

vxR_normalized = vxR / direction_magnitude
vyR_normalized = vyR / direction_magnitude
vzR_normalized = vzR / direction_magnitude

scale_factor = 1.5  # Adjust this factor as needed

spin_x_scaled = spin_x_normalized * scale_factor
spin_y_scaled = spin_y_normalized * scale_factor
spin_z_scaled = spin_z_normalized * scale_factor

# unit sphere calc
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))

# init 3d plot
fig = go.Figure()

# plot unit sphere (the ball)
fig.add_trace(go.Surface(x=x, y=y, z=z, opacity=0.1, colorscale='Blues', showscale=False))

# draw spin vector
fig.add_trace(go.Scatter3d(x=[-spin_x_scaled, spin_x_scaled], y=[-spin_y_scaled, spin_y_scaled], z=[-spin_z_scaled, spin_z_scaled], 
                           mode='lines', 
                           line=dict(color='red', width=5)))

# draw direction of pitch
fig.add_trace(go.Cone(x=[0], y=[0], z=[0], 
                      u=[vxR_normalized], v=[vyR_normalized], w=[vzR_normalized],
                      colorscale=[[0, 'green'], [1, 'green']],
                      showscale=False,
                      sizemode='absolute',
                      sizeref=0.3))  # Adjust sizeref to make the arrow skinnier

spin_x_round  = int(np.round(spin_x,0))
spin_y_round  = int(np.round(spin_y,0))
spin_z_round  = int(np.round(spin_z,0))

title_str = f'{pitcher_name}<br>{pitch_type}<br>w=<{spin_x_round},{spin_y_round},{spin_z_round}>'

fig.update_layout(scene=dict(
                    xaxis_title='X',
                    yaxis_title='Y',
                    zaxis_title='Z'),
                  title=title_str,
                  width=700, 
                  height=700)  

fig.show(fig)

### Write GIF

In [19]:
# Generate frames for the GIF
frames = []
for angle in range(0, 360, 10):
    camera = dict(
        eye=dict(x=2*np.sin(np.radians(angle)), y=2*np.cos(np.radians(angle)), z=0.5)
    )
    fig.update_layout(scene_camera=camera)
    fig.write_image(f"frame_{angle}.png")
    frames.append(imageio.imread(f"frame_{angle}.png"))

# Create GIF
imageio.mimsave('~/Desktop/rotating_plot.gif', frames, duration=0.1)